# Näide 01: Kiirvestlus OpenAI SDK kaudu

See märkmik näitab, kuidas kasutada OpenAI SDK-d koos Microsoft Foundry Localiga kiirvestluste jaoks.

## Ülevaade

See näide hõlmab:
- OpenAI Python SDK kasutamist koos Foundry Localiga
- Azure OpenAI ja kohalike Foundry konfiguratsioonide haldamist
- Vigade käsitlemise ja varuplaanide rakendamist
- FoundryLocalManageri kasutamist teenuste haldamiseks


## Eeltingimused

Veendu, et sul on:
1. Foundry Local installitud ja käivitatud
2. Mudel laaditud (nt `foundry model run phi-4-mini`)
3. Vajalikud Python'i paketid installitud

### Sõltuvuste installimine


In [ ]:
# Install required packages if not already installed
!pip install openai foundry-local-sdk

## Impordi vajalikud teegid


In [ ]:
import os
import sys
from openai import OpenAI

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, will use manual configuration")

## Konfiguratsioon

Seadista oma konfiguratsioon. Saad kasutada kas:
1. **Azure OpenAI** (pilvepõhine)
2. **Foundry Local** (kohalik järeldamine)

### Valik 1: Azure OpenAI konfiguratsioon


In [ ]:
# Azure OpenAI Configuration (uncomment and set your values)
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com"
# os.environ["AZURE_OPENAI_API_KEY"] = "your-api-key"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
# os.environ["MODEL"] = "your-deployment-name"

### Valik 2: Foundry kohalik konfiguratsioon


In [ ]:
# Foundry Local Configuration
MODEL_ALIAS = "phi-4-mini"  # Change this to your preferred model
BASE_URL = "http://localhost:8000"  # Default Foundry Local URL
API_KEY = ""  # Usually not needed for local

## Kliendi initsialiseerimine

Loo OpenAI klient vastavalt oma konfiguratsioonile:


In [ ]:
def create_client():
    """Create and configure OpenAI client."""
    
    # Check for Azure OpenAI configuration
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    azure_api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
    
    if azure_endpoint and azure_api_key:
        # Azure OpenAI path
        model = os.environ.get("MODEL", "your-deployment-name")
        client = OpenAI(
            base_url=f"{azure_endpoint}/openai",
            api_key=azure_api_key,
            default_query={"api-version": azure_api_version},
        )
        print(f"🌐 Using Azure OpenAI with model: {model}")
        return client, model
    
    # Foundry Local path
    if FOUNDRY_SDK_AVAILABLE:
        try:
            # Use FoundryLocalManager for proper service management
            manager = FoundryLocalManager(MODEL_ALIAS)
            model_info = manager.get_model_info(MODEL_ALIAS)
            
            # Configure OpenAI client to use local Foundry service
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key  # API key is not required for local usage
            )
            model = model_info.id
            print(f"🏠 Using Foundry Local SDK with model: {model}")
            return client, model
        except Exception as e:
            print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
    
    # Fallback to manual configuration
    client = OpenAI(
        base_url=f"{BASE_URL}/v1",
        api_key=API_KEY
    )
    model = MODEL_ALIAS
    print(f"🔧 Using manual configuration with model: {model}")
    return client, model

# Initialize the client
client, model = create_client()

## Lihtne vestluse näide

Testime lihtsat vestluse interaktsiooni:


In [ ]:
def chat_with_model(prompt, max_tokens=128):
    """Send a chat message to the model and get a response."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Test with a simple prompt
prompt = "Say hello from Foundry Local and explain what you are in one sentence."
print(f"👤 User: {prompt}")
print("\n🤖 Assistant:")
response = chat_with_model(prompt)
print(response)

## Interaktiivne vestlussessioon

Proovi erinevaid viise küsimuste esitamiseks, et näha, kuidas mudel reageerib:


In [ ]:
# Example prompts to try
example_prompts = [
    "What are the benefits of edge AI?",
    "Explain the difference between local and cloud AI inference.",
    "Write a short Python function to calculate the factorial of a number.",
    "What is Microsoft Foundry Local?"
]

for i, prompt in enumerate(example_prompts, 1):
    print(f"\n{'='*50}")
    print(f"Example {i}: {prompt}")
    print(f"{'='*50}")
    
    response = chat_with_model(prompt, max_tokens=200)
    print(response)

## Täiustatud kasutus: voogedastusega vastused

Pikemate vastuste puhul võib voogedastus pakkuda paremat kasutajakogemust:


In [ ]:
def chat_with_streaming(prompt, max_tokens=300):
    """Send a chat message with streaming response."""
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            stream=True
        )
        
        print("🤖 Assistant (streaming):")
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        print("\n")  # New line after streaming
        return full_response
    except Exception as e:
        return f"Error: {e}"

# Test streaming with a longer prompt
streaming_prompt = "Explain the advantages of running AI models locally on device versus in the cloud. Include privacy, latency, and cost considerations."
print(f"👤 User: {streaming_prompt}\n")
streaming_response = chat_with_streaming(streaming_prompt)

## Vigade käsitlemine ja diagnostika

Vaatame teenuse olekut ja saadaolevaid mudeleid:


In [ ]:
def check_service_health():
    """Check Foundry Local service health."""
    try:
        # Try to list available models
        models_response = client.models.list()
        models = [model.id for model in models_response.data]
        
        print("✅ Service is healthy")
        print(f"📋 Available models: {models}")
        print(f"🎯 Current model: {model}")
        
        if model in models:
            print("✅ Current model is available")
        else:
            print("⚠️ Current model may not be loaded")
            
    except Exception as e:
        print(f"❌ Service check failed: {e}")
        print("\n🔧 Troubleshooting tips:")
        print("1. Make sure Foundry Local is running")
        print("2. Check if a model is loaded: foundry service ps")
        print("3. Verify the endpoint URL is correct")

check_service_health()

## Kohandatud viipade testimine

Kasuta allolevat lahtrit, et testida oma viipasid:


In [ ]:
# Enter your custom prompt here
custom_prompt = "Write a haiku about artificial intelligence running on edge devices."

print(f"👤 User: {custom_prompt}\n")
custom_response = chat_with_model(custom_prompt, max_tokens=100)
print(f"🤖 Assistant: {custom_response}")

## Kokkuvõte

See märkmik näitas:

1. **✅ Kliendi seadistamine**: Kuidas konfigureerida OpenAI SDK Foundry Localiga
2. **✅ Lihtne vestlus**: Lihtsad päringu-vastuse interaktsioonid
3. **✅ Voogedastus**: Reaalajas vastuste genereerimine
4. **✅ Vigade käsitlemine**: Tõhus vigade käsitlemine ja diagnostika
5. **✅ Teenuse tervis**: Mudeli saadavuse ja teenuse oleku kontrollimine

### Järgmised sammud

- **Näidis 02**: Täiustatud SDK integratsioon Azure OpenAI toetusega
- **Näidis 04**: Chainlit vestlusrakenduse loomine
- **Näidis 05**: Mitme agendi orkestreerimissüsteemid
- **Näidis 06**: Nutikas mudeli suunamine

### Foundry Locali peamised eelised

- 🔒 **Privaatsus**: Andmed ei lahku kunagi teie seadmest
- ⚡ **Kiirus**: Madala latentsusega lokaalne järeldamine
- 💰 **Kulud**: API kasutuskulud puuduvad
- 🔌 **Võrguühenduseta**: Töötab ilma internetiühenduseta
- 🛠️ **Ühilduvus**: OpenAI-ga ühilduv API



---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti tõlgenduste eest.
